In [1]:
import numpy as np

import utils

NUM_EMBEDDING = 2000
def word2numpy(txt):
    return np.array([ord(c) for c in txt])

def wordlist2numpy(lines):
    return utils.pad_sequences([word2numpy(line) for line in lines],
                               maxlen=12, dtype=int, value=0)

RADICALS = ['.'] + list('אבגדהוזחטיכלמנסעפצקרשת') + ["ג'", "ז'", "צ'", 'שׂ']

BINYAN = 'פעל נפעל פיעל פועל הפעיל הופעל התפעל'.split()
TENSE = 'עבר הווה עתיד ציווי'.split()
VOICE = 'ראשון שני שלישי'.split()
GENDER = 'זכר נקבה'.split()
PLURAL = 'יחיד רבים'.split()

NAMES = ['B', 'T', 'V', 'G', 'P', 'R1', 'R2', 'R3', 'R4']
FEATURES = {
    'B': BINYAN,
    'T': TENSE,
    'V': VOICE,
    'G': GENDER,
    'P': PLURAL,
    'R1': RADICALS,
    'R2': RADICALS,
    'R3': RADICALS,
    'R4': RADICALS,
}

def to_category(name, b):
    return FEATURES[name].index(b)

def from_category(name, index):
    return FEATURES[name][index]

def list_to_category(name, bs):
    return np.array([to_category(name, b) for b in bs])

def list_from_category(name, indexes):
    return [from_category(name, index) for index in indexes]

def list_of_lists_to_category(items):
    return { name: list_to_category(name, item)
             for name, item in zip(NAMES, items) }

In [30]:
import torch

import torch.nn as nn
assert torch.cuda.is_available()

def to_device(d):
    if hasattr(d, 'cuda'):
        return d.cuda()
    return {k: v.cuda() for k, v in d.items()}

def create_model(UNITS):
    class Model(nn.Module):
        def __init__(self, UNITS):
            super().__init__()
            self.units = UNITS

            self.embed = nn.Embedding(num_embeddings=NUM_EMBEDDING, embedding_dim=UNITS)
            self.lstm1 = nn.LSTM(input_size=UNITS, hidden_size=UNITS, num_layers=1, batch_first=True, bidirectional=True)

            # {k: nn.Linear(in_features=UNITS, out_features=len(v)) for k, v in features.items()}
            self.binyan = nn.Linear(in_features=UNITS, out_features=len(BINYAN))
            self.tense = nn.Linear(in_features=UNITS, out_features=len(TENSE))
            self.voice = nn.Linear(in_features=UNITS, out_features=len(VOICE))
            self.gender = nn.Linear(in_features=UNITS, out_features=len(GENDER))
            self.plural = nn.Linear(in_features=UNITS, out_features=len(PLURAL))
            
            self.r1 = nn.Linear(in_features=UNITS, out_features=len(RADICALS))
            self.r2 = nn.Linear(in_features=UNITS, out_features=len(RADICALS))
            self.r3 = nn.Linear(in_features=UNITS, out_features=len(RADICALS))
            self.r4 = nn.Linear(in_features=UNITS, out_features=len(RADICALS))

            self.features = {
                'B': self.binyan,
                'T': self.tense,
                'V': self.voice,
                'G': self.gender,
                'P': self.plural,
                
                'R1': self.r1,
                'R2': self.r2,
                'R3': self.r3,
                'R4': self.r4,
            }

        def forward(self, x):
            embeds = self.embed(x)

            lstm_out, (h_n, c_n) = self.lstm1(embeds)
            left, right = torch.chunk(h_n, 2, dim=0)
            merge = torch.squeeze(left + right)

            outputs = { k: f(merge) for k, f in self.features.items() }
            return outputs

    model = Model(UNITS=UNITS)

    return to_device(model)


In [ ]:

wordlist = ['ידעתי', 'התאפס', 'יאבד']
binyanlist = ['פעל', 'התפעל', 'פיעל']

def sanity():
    with torch.no_grad():
        numpy_inp = wordlist2numpy(wordlist)
        inputs = to_device(torch.from_numpy(numpy_inp).to(torch.int64))
        tag_scores = model(inputs)
        expected = list_to_category(BINYAN, binyanlist)
        print(f'{tag_scores["B"].shape=}')
        print(f"{np.argmax(tag_scores['B'].cpu(), axis=1).shape=}")
        print(f"{expected.shape=}")

sanity()

In [3]:
import concrete

def load_dataset(filename):
    *features, verbs = concrete.load_dataset(filename)
    return wordlist2numpy(verbs), list_of_lists_to_category(features)

train = load_dataset('random_train.tsv')
valid = load_dataset('random_validate.tsv')

In [4]:
BATCH_SIZE = 64

def batch(a):
    ub = a.shape[0] // BATCH_SIZE * BATCH_SIZE
    return torch.from_numpy(a[:ub]).to(torch.int64).split(BATCH_SIZE)

def batch_all_ys(ys):
    res = []
    m = {k: batch(ys[k]) for k in NAMES}
    nbatches = len(m['B'])
    for i in range(nbatches):
        res.append({k: m[k][i] for k in NAMES})
    return res

def accuracy(output, ybatch):
    n = (ybatch != 0).sum()
    c = np.argmax(output, axis=1)
    return ((c == ybatch) & (c != 0)).sum() / n

def fit(model, x_train, y_train, x_valid, y_valid, *, epochs, criterion, optimizer):
    data = {
        'train': (batch(x_train), batch_all_ys(y_train)),
        'valid': (batch(x_valid), batch_all_ys(y_valid))
    }

    for epoch in range(epochs):
        for phase in ['train', 'valid']:
            total = len(data[phase][0])

            accs = {k: [] for k in NAMES}
            losses = []

            for i, (inputs, labels) in enumerate(zip(*data[phase])):
                inputs = to_device(inputs)
                
                if phase == 'train':
                    model.train()
                    outputs = model(inputs)
                else:
                    model.eval()
                    with torch.no_grad():
                        outputs = model(inputs)
                    
                outputs = to_device(outputs)
                labels = to_device(labels)

                loss = sum(criterion(outputs[k], labels[k]) for k in outputs)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                outputs = {k: v.cpu().data.numpy() for k, v in outputs.items()}
                labels = {k: v.cpu().data.numpy() for k, v in labels.items()}

                for k in outputs:
                    accs[k].append(accuracy(outputs[k], labels[k]))

                losses.append(loss.item())
                RUNSIZE = 100
                if phase == 'valid' and i >= total-1 or i % RUNSIZE == RUNSIZE - 1:
                    print("{:2} {:4}/{:4}".format(epoch, i, total), end=' ')
                    for k in accs:
                        print("{}_acc: {:.4f}".format(k, np.mean(accs[k])), end=' ')
                    print("Loss: {:.4f}".format(np.mean(losses)), end='\r')
                    accs = {k: [] for k in NAMES}
                    losses = []
            print()

@torch.no_grad()
def predict(model, *verbs):
    model.eval()
    verbs = wordlist2numpy(verbs)
    verbs = to_device(torch.from_numpy(verbs).to(torch.int64))
    outputs = model(verbs)
    res = {k: from_category(k, torch.argmax(v))
           for k, v in outputs.items()}
    res['R'] = ''.join(res[k] for k in ['R1', 'R2', 'R3', 'R4']).replace('.', '')
    return res
    

In [31]:
model = create_model(UNITS=100)

print(model)
fit(model,
    *train,
    *valid,
    epochs=3,
    criterion=nn.CrossEntropyLoss(),  # add ignore_index for root
    optimizer=torch.optim.Adam(model.parameters(), lr=1e-3)
)

Model(
  (embed): Embedding(2000, 100)
  (lstm1): LSTM(100, 100, batch_first=True, bidirectional=True)
  (binyan): Linear(in_features=100, out_features=7, bias=True)
  (tense): Linear(in_features=100, out_features=4, bias=True)
  (voice): Linear(in_features=100, out_features=3, bias=True)
  (gender): Linear(in_features=100, out_features=2, bias=True)
  (plural): Linear(in_features=100, out_features=2, bias=True)
  (r1): Linear(in_features=100, out_features=27, bias=True)
  (r2): Linear(in_features=100, out_features=27, bias=True)
  (r3): Linear(in_features=100, out_features=27, bias=True)
  (r4): Linear(in_features=100, out_features=27, bias=True)
)
 0 1499/1562 B_acc: 0.8249 T_acc: 0.9036 V_acc: 0.6375 G_acc: 0.7487 P_acc: 0.9803 R1_acc: 0.9198 R2_acc: 0.7978 R3_acc: 0.7497 R4_acc: 0.9384 Loss: 3.23142
 0  155/ 156 B_acc: 0.8248 T_acc: 0.9299 V_acc: 0.6064 G_acc: 0.7594 P_acc: 0.9642 R1_acc: 0.9235 R2_acc: 0.7863 R3_acc: 0.7616 R4_acc: 0.9456 Loss: 3.1337
 1 1499/1562 B_acc: 0.8563 T_

In [32]:
print(predict(model, 'סבסו'))
print(predict(model, 'מקדו'))
print(predict(model, 'נמזר'))
print(predict(model, 'כרדו'))

{'B': 'פעל', 'T': 'ציווי', 'V': 'שני', 'G': 'זכר', 'P': 'רבים', 'R1': 'ס', 'R2': 'ב', 'R3': '.', 'R4': 'ס', 'R': 'סבס'}
{'B': 'פיעל', 'T': 'ציווי', 'V': 'שלישי', 'G': 'זכר', 'P': 'רבים', 'R1': 'מ', 'R2': 'ד', 'R3': '.', 'R4': 'ד', 'R': 'מדד'}
{'B': 'פיעל', 'T': 'עתיד', 'V': 'ראשון', 'G': 'זכר', 'P': 'רבים', 'R1': 'מ', 'R2': 'ז', 'R3': '.', 'R4': 'ר', 'R': 'מזר'}
{'B': 'פעל', 'T': 'עבר', 'V': 'שלישי', 'G': 'זכר', 'P': 'רבים', 'R1': 'כ', 'R2': 'ד', 'R3': '.', 'R4': 'ד', 'R': 'כדד'}


In [7]:
print(predict(model, 'הבריל'))
print(predict(model, 'חגוו'))
print(predict(model, 'עגו'))
print(predict(model, 'צירלל'))

{'B': 'הפעיל', 'T': 'עבר', 'V': 'שלישי', 'G': 'נקבה', 'P': 'יחיד', 'R1': 'ב', 'R2': 'ר', 'R3': '.', 'R4': 'ל', 'R': 'ברל'}
{'B': 'פעל', 'T': 'עבר', 'V': 'שלישי', 'G': 'זכר', 'P': 'רבים', 'R1': 'ח', 'R2': 'ג', 'R3': '.', 'R4': 'י', 'R': 'חגי'}
{'B': 'פעל', 'T': 'עבר', 'V': 'שלישי', 'G': 'נקבה', 'P': 'רבים', 'R1': 'ע', 'R2': 'ג', 'R3': '.', 'R4': 'י', 'R': 'עגי'}
{'B': 'פיעל', 'T': 'עבר', 'V': 'שלישי', 'G': 'זכר', 'P': 'יחיד', 'R1': 'צ', 'R2': 'ר', 'R3': 'ל', 'R4': 'ל', 'R': 'צרלל'}


In [28]:
print(predict(model, "התנגן"))

{'B': 'התפעל', 'T': 'ציווי', 'V': 'שני', 'G': 'זכר', 'P': 'יחיד', 'R1': 'נ', 'R2': 'ג', 'R3': '.', 'R4': 'נ', 'R': 'נגנ'}


In [22]:
print(predict(model, "משקה"))

{'B': 'הפעיל', 'T': 'הווה', 'V': 'שלישי', 'G': 'נקבה', 'P': 'יחיד', 'R1': 'ש', 'R2': 'ק', 'R3': '.', 'R4': 'י', 'R': 'שקי'}
